# Create dataset
We start considering the Macrozone NORD

In [87]:
import sys
print(sys.executable)

c:\Users\samueledelia\.conda\envs\myenv\python.exe


In [88]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Add the source directory to the system path
sys.path.append(os.path.abspath('../src'))
from open_data import fetch_db_table_sqlserver16
import utils
import importlib

# Reload the module
importlib.reload(utils)

# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_SEGNO_QH" 
qh = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
qh = qh.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
qh = qh[['MACROZONA', 'ORAINI', 'SBIL_MWH']]
# Convert ORAINI to datetime using the correct method
qh['ORAINI'] = pd.to_datetime(qh['ORAINI'], format='%Y%m%d%H%M')
qh.set_index('ORAINI', inplace=True)
# Nord
qh_nord = qh[qh['MACROZONA'] == 'NORD']
qh_nord = qh_nord.drop(columns = ['MACROZONA'])

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


In [89]:
# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_ENERGY_BALANCE" 
power = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
power = power.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
power = power[['ENERGY_BALANCE_GWh', 'ORAINI', 'ENERGY_SOURCE']]
# Convert ORAINI to datetime using the correct method
power['ORAINI'] = pd.to_datetime(power['ORAINI'], format='%Y%m%d%H%M')
power.set_index('ORAINI', inplace=True)
thermo = utils.filter_and_rename_energy_source(power, 'Thermal', 'THERMO_GWH')
wind = utils.filter_and_rename_energy_source(power, 'Wind', 'WIND_GWH')
solar = utils.filter_and_rename_energy_source(power, 'Photovoltaic', 'SOLAR_GWH')
hydro = utils.filter_and_rename_energy_source(power, 'Hydro', 'HYDRO_GWH')
geothermal = utils.filter_and_rename_energy_source(power, 'Geothermal', 'GEO_GWH')
forex = utils.filter_and_rename_energy_source(power, 'Net Foreign Exchange', 'FOREX_GWH')

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


In [90]:
# Resample the data to 15-minute intervals and divide by 4
wind = wind.resample('15min').ffill()  # Forward fill to maintain hourly data for the quarter-hour 
wind['WIND_GWH'] = wind['WIND_GWH'] / 4
wind['WIND_MWH'] = wind['WIND_GWH'] * 1000
# Drop the original hourly GWh column
wind = wind.drop(columns=['WIND_GWH'])

wind

,WIND_MWH
ORAINI,
2019-01-01 00:00:00,1275.0
2019-01-01 00:15:00,1275.0
2019-01-01 00:30:00,1275.0
2019-01-01 00:45:00,1275.0
2019-01-01 01:00:00,1257.5
...,...
2024-11-03 22:00:00,257.5
2024-11-03 22:15:00,257.5
2024-11-03 22:30:00,257.5


In [91]:
# Resample the data to 15-minute intervals and divide by 4
solar = solar.resample('15min').ffill()  # Forward fill to maintain hourly data for the quarter-hour 
solar['SOLAR_GWH'] = solar['SOLAR_GWH'] / 4
solar['SOLAR_MWH'] = solar['SOLAR_GWH'] * 1000
# Drop the original hourly GWh column
solar = solar.drop(columns=['SOLAR_GWH'])

solar

,SOLAR_MWH
ORAINI,
2019-01-01 00:00:00,0.0
2019-01-01 00:15:00,0.0
2019-01-01 00:30:00,0.0
2019-01-01 00:45:00,0.0
2019-01-01 01:00:00,0.0
...,...
2024-11-03 22:00:00,0.0
2024-11-03 22:15:00,0.0
2024-11-03 22:30:00,0.0


In [92]:
# Resample the data to 15-minute intervals and divide by 4
thermo = thermo.resample('15min').ffill()  # Forward fill to maintain hourly data for the quarter-hour 
thermo['THERMO_GWH'] = thermo['THERMO_GWH'] / 4
thermo['THERMO_MWH'] = thermo['THERMO_GWH'] * 1000
# Drop the original hourly GWh column
thermo = thermo.drop(columns=['THERMO_GWH'])

thermo

,THERMO_MWH
ORAINI,
2019-01-01 00:00:00,3337.5
2019-01-01 00:15:00,3337.5
2019-01-01 00:30:00,3337.5
2019-01-01 00:45:00,3337.5
2019-01-01 01:00:00,3272.5
...,...
2024-11-03 22:00:00,2700.0
2024-11-03 22:15:00,2700.0
2024-11-03 22:30:00,2700.0


In [93]:
# Resample the data to 15-minute intervals and divide by 4
hydro = hydro.resample('15min').ffill()  # Forward fill to maintain hourly data for the quarter-hour 
hydro['HYDRO_GWH'] = hydro['HYDRO_GWH'] / 4
hydro['HYDRO_MWH'] = hydro['HYDRO_GWH'] * 1000
# Drop the original hourly GWh column
hydro = hydro.drop(columns=['HYDRO_GWH'])

hydro

,HYDRO_MWH
ORAINI,
2019-01-01 00:00:00,515.0
2019-01-01 00:15:00,515.0
2019-01-01 00:30:00,515.0
2019-01-01 00:45:00,515.0
2019-01-01 01:00:00,437.5
...,...
2024-11-03 22:00:00,1270.0
2024-11-03 22:15:00,1270.0
2024-11-03 22:30:00,1270.0


In [94]:
# Resample the data to 15-minute intervals and divide by 4
geothermal = geothermal.resample('15min').ffill()  # Forward fill to maintain hourly data for the quarter-hour 
geothermal['GEO_GWH'] = geothermal['GEO_GWH'] / 4
geothermal['GEO_MWH'] = geothermal['GEO_GWH'] * 1000
# Drop the original hourly GWh column
geothermal = geothermal.drop(columns=['GEO_GWH'])

geothermal

,GEO_MWH
ORAINI,
2019-01-01 00:00:00,167.5
2019-01-01 00:15:00,167.5
2019-01-01 00:30:00,167.5
2019-01-01 00:45:00,167.5
2019-01-01 01:00:00,167.5
...,...
2024-11-03 22:00:00,147.5
2024-11-03 22:15:00,147.5
2024-11-03 22:30:00,147.5


In [95]:
## Resample the data to 15-minute intervals and divide by 4
#forex = forex.resample('15min').ffill()  # Forward fill to maintain hourly data for the quarter-hour 
#forex['FOREX_GWH'] = forex['FOREX_GWH'] / 4
#forex['FOREX_MWH'] = forex['FOREX_GWH'] * 1000
# Drop the original hourly GWh column
#forex = forex.drop(columns=['FOREX_GWH'])

#forex

In [96]:
# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_PHYSICAL_FOREIGN_FLOW" 
physical_forex = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
physical_forex = physical_forex.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
physical_forex = physical_forex[['PHYSICAL_FOREIGN_FLOW_MW', 'ORAINI', 'COUNTRY']]
# Convert ORAINI to datetime using the correct method
physical_forex['ORAINI'] = pd.to_datetime(physical_forex['ORAINI'], format='%Y%m%d%H%M')
physical_forex.set_index('ORAINI', inplace=True)

physical_forex

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,PHYSICAL_FOREIGN_FLOW_MW,COUNTRY
ORAINI,,
2019-01-01 00:00:00,156.0,Austria
2019-01-01 00:00:00,-142.0,Corsica
2019-01-01 00:00:00,-321.0,France
2019-01-01 00:00:00,-313.0,Greece
2019-01-01 00:00:00,-80.0,Malta
...,...,...
2024-11-03 23:00:00,2782.0,France
2024-11-03 23:00:00,0.0,Greece
2024-11-03 23:00:00,-68.0,Malta


In [97]:
austria_physical = utils.process_foreign_exchange(physical_forex, 'Austria')
corsica_physical = utils.process_foreign_exchange(physical_forex, 'Corsica')
france_physical = utils.process_foreign_exchange(physical_forex, 'France')
greece_physical = utils.process_foreign_exchange(physical_forex, 'Greece')
malta_physical = utils.process_foreign_exchange(physical_forex, 'Malta')
slovenia_physical = utils.process_foreign_exchange(physical_forex, 'Slovenia')
switzerland_physical = utils.process_foreign_exchange(physical_forex, 'Switzerland')
montenegno_physical = utils.process_foreign_exchange(physical_forex, 'Montenegno')

In [98]:
# SQL query to fetch data
sql_query = "SELECT * FROM GME_MGP_MI_QUANTITA" 
volumes = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
volumes = volumes.sort_values(by=['FLOWDATE','FLOWHOUR'])
# Convert FLOWDATE to a string and then to datetime (YYYYMMDD format)
volumes['FLOWDATE'] = pd.to_datetime(volumes['FLOWDATE'].astype(str), format='%Y%m%d')
# Subtracting one hour from FLOWHOUR
volumes['FLOWHOUR'] = volumes['FLOWHOUR'] - 1
# Convert FLOWHOUR to timedelta (number of hours) and add it to FLOWDATE
volumes['ORAINI'] = volumes['FLOWDATE'] + pd.to_timedelta(volumes['FLOWHOUR'], unit='h')
# Dropping the old FLOWDATE and FLOWHOUR columns
volumes = volumes.drop(columns=['FLOWDATE', 'FLOWHOUR'])
volumes.set_index('ORAINI', inplace=True)

mgp_volumes = volumes[volumes['MARKET'] == 'MGP']
mgp_volumes = mgp_volumes.drop(columns=['MARKET'])

mgp_volumes

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,TOTAL_PURCHASES,NAT_PURCHASES,CNOR_PURCHASES,CSUD_PURCHASES,NORD_PURCHASES,SARD_PURCHASES,SICI_PURCHASES,SUD_PURCHASES,AUST_PURCHASES,BRNN_PURCHASES,...,ROSN_SALES,SLOV_SALES,SVIZ_SALES,BSP_SALES,MALT_SALES,XAUS_SALES,XFRA_SALES,TOTITABSP_SALES,TOTITABSP_PURCHASES,LOADTIME
ORAINI,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,23521.449,24243.696,2372.707,4174.117,11460.324,901.738,1679.169,2148.394,0.0,None,...,1843.786,0.0,2438.0,0.0,0.0,187.0,0.0,24056.696,23730.449,2018-12-31 13:51:37.097
2019-01-01 01:00:00,22417.488,23396.353,2245.300,3885.330,10964.464,861.702,1571.783,2015.909,0.0,None,...,1845.050,0.0,2342.0,0.0,0.0,187.0,0.0,23209.353,22588.488,2018-12-31 13:51:37.097
2019-01-01 02:00:00,21496.678,22353.623,2145.655,3606.687,10604.767,829.694,1474.098,1888.777,0.0,None,...,1845.647,0.0,2175.0,0.0,0.0,0.0,0.0,22353.623,22057.240,2018-12-31 13:51:37.097
2019-01-01 03:00:00,20322.418,20576.217,2050.474,3392.350,10234.366,805.191,1401.924,1804.113,0.0,None,...,983.111,0.0,2310.0,0.0,0.0,0.0,0.0,20576.217,20468.106,2018-12-31 13:51:37.100
2019-01-01 04:00:00,19979.994,21238.994,2016.645,3314.058,10088.680,791.251,1366.832,1769.976,0.0,None,...,978.775,0.0,2351.0,433.0,0.0,187.0,0.0,21051.994,19979.994,2018-12-31 13:51:37.100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-05 19:00:00,39040.934,39532.934,3297.935,6995.922,21618.525,948.726,2139.919,2272.374,0.0,None,...,NaN,0.0,1754.0,0.0,0.0,0.0,736.0,38499.411,39397.934,2024-11-04 13:10:40.477
2024-11-05 20:00:00,36667.998,37150.607,3051.745,6540.158,20468.487,924.273,2056.293,2165.545,0.0,None,...,NaN,0.0,2023.0,0.0,0.0,0.0,1861.0,35137.081,37015.998,2024-11-04 13:10:40.477
2024-11-05 21:00:00,33536.127,34290.241,2776.385,5976.733,18816.603,859.662,1953.161,2010.393,0.0,None,...,NaN,0.0,2235.0,0.0,0.0,0.0,2937.0,30888.582,34190.627,2024-11-04 13:10:40.477


In [99]:
mgp_volumes_nord = mgp_volumes[['NORD_PURCHASES', 'NORD_SALES']].copy()
mgp_volumes_nord.rename(columns={"NORD_PURCHASES": "MGP_NORD_PURCHASES", "NORD_SALES": "MGP_NORD_SALES"}, inplace=True)
mgp_volumes_nord = mgp_volumes_nord[~mgp_volumes_nord.index.duplicated(keep='first')]
mgp_volumes_nord = mgp_volumes_nord.resample('15min').ffill()/4


mi1_volumes = volumes[volumes['MARKET'] == 'MI1']
mi1_volumes = mi1_volumes.drop(columns=['MARKET'])
mi1_volumes_nord = mi1_volumes[['NORD_PURCHASES', 'NORD_SALES']].copy()
mi1_volumes_nord.rename(columns={"NORD_PURCHASES": "MI1_NORD_PURCHASES", "NORD_SALES": "MI1_NORD_SALES"}, inplace=True)
mi1_volumes_nord = mi1_volumes_nord[~mi1_volumes_nord.index.duplicated(keep='first')]
mi1_volumes_nord = mi1_volumes_nord.resample('15min').ffill()/4

In [100]:
# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_SCHEDULED_INTERNAL_EXCHANGE"
scheduled_internal = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
scheduled_internal = scheduled_internal.sort_values(by='ORAINI')

# Filter the DataFrame for the relevant columns and rows
scheduled_internal = scheduled_internal[['SCHEDULED_INTERNAL_EXCHANGE_MW', 'ORAINI', 'FLOW_FROM', 'FLOW_TO']]
# Convert ORAINI to datetime using the correct method
scheduled_internal['ORAINI'] = pd.to_datetime(scheduled_internal['ORAINI'], format='%Y%m%d%H%M')
scheduled_internal.set_index('ORAINI', inplace=True)
scheduled_internal = scheduled_internal[(scheduled_internal['FLOW_FROM'] == 'CNOR') & (scheduled_internal['FLOW_TO'] == 'NORD') ]
scheduled_internal = scheduled_internal.drop(columns=['FLOW_FROM','FLOW_TO'])
scheduled_internal_nord = scheduled_internal.resample('15min').ffill()/4

scheduled_internal_nord

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,SCHEDULED_INTERNAL_EXCHANGE_MW
ORAINI,
2019-01-01 00:00:00,158.7875
2019-01-01 00:15:00,158.7875
2019-01-01 00:30:00,158.7875
2019-01-01 00:45:00,158.7875
2019-01-01 01:00:00,235.1475
...,...
2024-11-02 12:00:00,72.9300
2024-11-02 12:15:00,72.9300
2024-11-02 12:30:00,72.9300


In [101]:
# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_TOTAL_LOAD_QH" 
total_load = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
total_load = total_load.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
total_load = total_load[['ZONA', 'TOTAL_LOAD_MW', 'FORECAST_TOTAL_LOAD_MW', 'ORAINI']]
# Convert ORAINI to datetime using the correct method
total_load['ORAINI'] = pd.to_datetime(total_load['ORAINI'], format='%Y%m%d%H%M')
total_load.set_index('ORAINI', inplace=True)
subset_zonas = ['NORD']
mnord_load = utils.filter_load_by_zona(total_load, subset_zonas)
mnord_load = mnord_load.drop(columns=['ZONA'])
mnord_load

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,TOTAL_LOAD_MW,FORECAST_TOTAL_LOAD_MW
ORAINI,,
2019-01-01 00:00:00,11776.405,12085.191
2019-01-01 00:15:00,11723.369,12010.132
2019-01-01 00:30:00,11771.133,11984.692
2019-01-01 00:45:00,11740.606,11935.066
2019-01-01 01:00:00,11559.374,11825.838
...,...,...
2024-11-03 22:45:00,12487.711,12550.347
2024-11-03 23:00:00,12300.335,12336.928
2024-11-03 23:15:00,11967.276,12040.106


Total and forecast load ENTSOE

In [102]:
# Load the data
df_load_ensoe_path = r'C:\imbalance_forecast\data\df_load_entsoe.csv'
df_load_ensoe_path = pd.read_csv(df_load_ensoe_path, index_col=0, parse_dates=True)

# Access the relevant columns
mnord_load_entsoe = df_load_ensoe_path[['DAY_AHEAD_TOTAL_LOAD_MW_NORD', 'ACTUAL_TOTAL_LOAD_MW_NORD']]

# Check for duplicate timestamps
duplicates = mnord_load_entsoe.index.duplicated(keep=False)

if duplicates.any():
    # Handle duplicates: average values for duplicate timestamps
    mnord_load_entsoe = mnord_load_entsoe.groupby(mnord_load_entsoe.index).mean()

# Now resample the data
mnord_load_entsoe = mnord_load_entsoe.resample('15min').ffill() / 4

# Display the result
mnord_load_entsoe

,DAY_AHEAD_TOTAL_LOAD_MW_NORD,ACTUAL_TOTAL_LOAD_MW_NORD
ORAINI,,
2019-01-01 00:00:00,2777.50,2877.75
2019-01-01 00:15:00,2777.50,2877.75
2019-01-01 00:30:00,2777.50,2877.75
2019-01-01 00:45:00,2777.50,2877.75
2019-01-01 01:00:00,2668.75,2759.75
...,...,...
2024-11-04 11:00:00,NaN,NaN
2024-11-04 11:15:00,NaN,NaN
2024-11-04 11:30:00,NaN,NaN


Lag SBIL

In [103]:
lags = [-1, -4, -96]

# Create a new DataFrame with SBIL_MWH and its lagged values
df_sbil_lagged = pd.DataFrame(qh_nord['SBIL_MWH'])

# Add lagged columns
for lag in lags:
    df_sbil_lagged[f'SBIL_MWH_lag{abs(lag)}'] = df_sbil_lagged['SBIL_MWH'].shift(-lag)

df_sbil_lagged = df_sbil_lagged.drop('SBIL_MWH', axis=1)
df_sbil_lagged

,SBIL_MWH_lag1,SBIL_MWH_lag4,SBIL_MWH_lag96
ORAINI,,,
2019-01-01 00:00:00,NaN,NaN,NaN
2019-01-01 00:15:00,-47.312,NaN,NaN
2019-01-01 00:30:00,-30.343,NaN,NaN
2019-01-01 00:45:00,-20.311,NaN,NaN
2019-01-01 01:00:00,-13.934,-47.312,NaN
...,...,...,...
2024-11-04 14:15:00,238.566,0.358,283.118
2024-11-04 14:30:00,213.311,-97.518,185.808
2024-11-04 14:45:00,-30.007,-104.940,84.808


In [107]:
from functools import reduce
# List of all the DataFrames to be merged
dataframes = [qh_nord, df_sbil_lagged, wind, thermo, solar, hydro, geothermal, austria_physical, france_physical, slovenia_physical, switzerland_physical, scheduled_internal_nord, mgp_volumes_nord, mi1_volumes_nord, mnord_load, mnord_load_entsoe]
# Use reduce to merge all DataFrames on 'ORAINI'
df_nord = reduce(lambda left, right: pd.merge(left, right, on='ORAINI', how='outer'), dataframes)

df_nord = df_nord[df_nord.index <= '2024-03-11']
df_nord = df_nord[df_nord.index >= '2019-01-03']
df_nord.head(35)

,SBIL_MWH,SBIL_MWH_lag1,SBIL_MWH_lag4,SBIL_MWH_lag96,WIND_MWH,THERMO_MWH,SOLAR_MWH,HYDRO_MWH,GEO_MWH,AUSTRIA_MWQH,...,SWITZERLAND_MWQH,SCHEDULED_INTERNAL_EXCHANGE_MW,MGP_NORD_PURCHASES,MGP_NORD_SALES,MI1_NORD_PURCHASES,MI1_NORD_SALES,TOTAL_LOAD_MW,FORECAST_TOTAL_LOAD_MW,DAY_AHEAD_TOTAL_LOAD_MW_NORD,ACTUAL_TOTAL_LOAD_MW_NORD
ORAINI,,,,,,,,,,,,,,,,,,,,,
2019-01-03 00:00:00,-1.847,63.465,-156.567,38.978,1402.5,3242.5,0.0,407.5,167.5,39.25,...,526.00,82.8000,3221.22925,2003.79250,151.94800,106.55450,13261.060,13912.337,3194.50,3126.25
2019-01-03 00:15:00,-11.944,-1.847,-63.103,56.932,1402.5,3242.5,0.0,407.5,167.5,39.25,...,526.00,82.8000,3221.22925,2003.79250,151.94800,106.55450,12996.362,13697.814,3194.50,3126.25
2019-01-03 00:30:00,0.336,-11.944,16.441,122.707,1402.5,3242.5,0.0,407.5,167.5,39.25,...,526.00,82.8000,3221.22925,2003.79250,151.94800,106.55450,12869.871,13592.390,3194.50,3126.25
2019-01-03 00:45:00,55.911,0.336,63.465,174.893,1402.5,3242.5,0.0,407.5,167.5,39.25,...,526.00,82.8000,3221.22925,2003.79250,151.94800,106.55450,12624.143,13419.621,3194.50,3126.25
2019-01-03 01:00:00,-17.541,55.911,-1.847,99.216,1360.0,3160.0,0.0,395.0,170.0,40.50,...,398.75,221.8950,3076.62150,1992.72250,129.96975,134.68950,12234.627,13132.506,2954.75,2918.50
2019-01-03 01:15:00,-46.081,-17.541,-11.944,104.522,1360.0,3160.0,0.0,395.0,170.0,40.50,...,398.75,221.8950,3076.62150,1992.72250,129.96975,134.68950,12286.212,12814.419,2954.75,2918.50
2019-01-03 01:30:00,-21.225,-46.081,0.336,130.053,1360.0,3160.0,0.0,395.0,170.0,40.50,...,398.75,221.8950,3076.62150,1992.72250,129.96975,134.68950,12437.186,12880.788,2954.75,2918.50
2019-01-03 01:45:00,29.607,-21.225,55.911,136.617,1360.0,3160.0,0.0,395.0,170.0,40.50,...,398.75,221.8950,3076.62150,1992.72250,129.96975,134.68950,12243.895,12746.228,2954.75,2918.50
2019-01-03 02:00:00,-46.653,29.607,-17.541,106.558,1220.0,3150.0,0.0,405.0,170.0,43.25,...,388.00,298.3425,2990.44050,1977.44700,108.58025,106.28475,12027.606,12552.932,2827.25,2825.25


In [105]:
df_nord.tail(25)

,SBIL_MWH,SBIL_MWH_lag1,SBIL_MWH_lag4,SBIL_MWH_lag96,WIND_MWH,THERMO_MWH,SOLAR_MWH,HYDRO_MWH,GEO_MWH,AUSTRIA_MWQH,...,SWITZERLAND_MWQH,SCHEDULED_INTERNAL_EXCHANGE_MW,MGP_NORD_PURCHASES,MGP_NORD_SALES,MI1_NORD_PURCHASES,MI1_NORD_SALES,TOTAL_LOAD_MW,FORECAST_TOTAL_LOAD_MW,DAY_AHEAD_TOTAL_LOAD_MW_NORD,ACTUAL_TOTAL_LOAD_MW_NORD
ORAINI,,,,,,,,,,,,,,,,,,,,,
2024-03-10 18:00:00,154.907,-24.060,195.264,-104.128,1440.0,2275.0,2.5,1727.5,155.0,62.50,...,1147.50,46.305,4211.55575,2585.89450,146.23825,203.99550,17952.554,17896.750,4284.75,4339.00
2024-03-10 18:15:00,57.474,154.907,246.132,104.844,1440.0,2275.0,2.5,1727.5,155.0,62.50,...,1147.50,46.305,4211.55575,2585.89450,146.23825,203.99550,18717.886,18559.667,4284.75,4339.00
2024-03-10 18:30:00,-114.593,57.474,65.446,81.467,1440.0,2275.0,2.5,1727.5,155.0,62.50,...,1147.50,46.305,4211.55575,2585.89450,146.23825,203.99550,19556.676,19124.851,4284.75,4339.00
2024-03-10 18:45:00,-207.391,-114.593,-24.060,-90.002,1440.0,2275.0,2.5,1727.5,155.0,62.50,...,1147.50,46.305,4211.55575,2585.89450,146.23825,203.99550,20089.395,19615.101,4284.75,4339.00
2024-03-10 19:00:00,-215.010,-207.391,154.907,-172.590,1325.0,2285.0,0.0,2225.0,155.0,57.50,...,1257.00,46.305,4371.02225,2797.34600,184.84675,319.42550,20399.797,19755.141,4414.00,4587.25
2024-03-10 19:15:00,-211.963,-215.010,57.474,-151.080,1325.0,2285.0,0.0,2225.0,155.0,57.50,...,1257.00,46.305,4371.02225,2797.34600,184.84675,319.42550,20397.057,19928.699,4414.00,4587.25
2024-03-10 19:30:00,-125.881,-211.963,-114.593,-34.305,1325.0,2285.0,0.0,2225.0,155.0,57.50,...,1257.00,46.305,4371.02225,2797.34600,184.84675,319.42550,20394.751,20049.743,4414.00,4587.25
2024-03-10 19:45:00,-36.376,-125.881,-207.391,-73.075,1325.0,2285.0,0.0,2225.0,155.0,57.50,...,1257.00,46.305,4371.02225,2797.34600,184.84675,319.42550,19926.332,19725.274,4414.00,4587.25
2024-03-10 20:00:00,-201.162,-36.376,-215.010,-2.605,1172.5,2297.5,0.0,2045.0,155.0,61.25,...,1263.75,46.305,4150.92400,2691.02975,152.27550,341.06900,19750.276,19559.139,4238.25,4400.50


In [106]:
from pathlib import Path 
filepath = Path('C:\imbalance_forecast\data\df_nord.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
df_nord.to_csv(filepath)


<>:2: SyntaxWarning: invalid escape sequence '\i'
<>:2: SyntaxWarning: invalid escape sequence '\i'
C:\Users\samueledelia\AppData\Local\Temp\3\ipykernel_2628\4209121318.py:2: SyntaxWarning: invalid escape sequence '\i'
  filepath = Path('C:\imbalance_forecast\data\df_nord.csv')
